<a href="https://colab.research.google.com/github/haelyak/ODScheduler/blob/main/ODScheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# OD Scheduler

### This is a Python script that will create an on-duty scheduler for a single cabin with four counselors.

Before running the script, the user will need to input the rest_night_periods. This is a list of rest and night periods where rests are differentiated by the letter R and nights are differentiated by the letter N. The numbers represent the number day of a session. For instance R1 would represent the rest period on the first day of the session. If a given day does not have a rest period, for instance day 7 might be a trip day and only have a night period, you would skip R7 in the list and input N7 only. Similarily, if a night is an ALL-ON night, you would skip that night in the list.

Once the rest_night_periods are input, the user will need to adjust the counselors_off_days. This is a dictionary where the keys are the counselors and the values are a list of numbers which represent the days off a counselor has. For instance if counselor A1 is off on day 3 and day 10 in the session, the dictionary would contain "Counselor A1" : [3, 10]. If multiple counselors have the same off day, differentiate the counselors by a number 1 or 2 (ie A1 and A2).

After inputting the rest_night_periods and counslors_off_days, you can click the triangle play symbol in the upper right-hand corner of the code box to run the script.

The script will create a printout at the bottom of the code box. The top of the printout will calculate how many rests, nights, and total assignments a counselor has. It will flag if one counselor has significantly more assignments than others in the cabin.

The script will also create a "Final Schedule" that will assign a counselor to each rest and night period in the session. You will need this Final Schedule to create the pattern templates at the top of the OD schedule for the DHs.

In [4]:
from collections import defaultdict

# User Input

# Schedule of rest and night periods where the number is the number day of a session
rest_night_periods = ['R1', 'N1', 'R2', 'N2', 'R3', 'N3', 'R4', 'N4', 'R5', 'N5',
                      'R6', 'N6', 'N7', 'R8', 'N8', 'R9', 'N9', 'R10', 'N10',
                      'R11', 'N11', 'R12', 'N12', 'N13', 'R14', 'N14', 'R15', 'N15',
                      'R16', 'N16', 'R17', 'N17', 'R18', 'N18', 'R19', 'N19', 'R20']

# List of days that counselors are not on duty
# Differentiate counselors that have the same off days by 1 or 2
counselors_off_days = {
    'Counselor A1': [3, 10],
    'Counselor B1': [5, 12],
    'Counselor C1': [7, 13],
    'Counselor C2': [7, 13]
}

# Function to check if a counselor can be assigned to a specific period
def can_assign(counselor, period, day, assignments):
    # Check if the counselor is off on the given day
    if day in counselors_off_days[counselor]:
        return False, f"{counselor} is off on day {day}"
    # Check if the counselor is off the day after (for night shifts)
    if period.startswith('N') and (day + 1) in counselors_off_days[counselor]:
        return False, f"{counselor} is off the day after (day {day + 1})"
    # Check if the counselor is already assigned on the same day
    if f'R{day}' in assignments[counselor] or f'N{day}' in assignments[counselor]:
        return False, f"{counselor} is already assigned to a period on day {day}"
    # Check if the counselor is already assigned to a night shift the day before or the day after
    if period.startswith('N') and (f'N{day - 1}' in assignments[counselor] or f'N{day + 1}' in assignments[counselor]):
        return False, f"{counselor} is already assigned to a night shift the day before or the day after day {day}"
    return True, ""

# Backtracking function to assign counselors to periods
def assign_periods(periods, assignments, counselors, rest_count, night_count, total_count, period_index=0):
    if period_index == len(periods):  # Base Case: finished all periods
        return True

    period = periods[period_index]
    day = int(period[1:])
    period_type = 'R' if period.startswith('R') else 'N'

    # Track tried assignments to avoid redundant checks
    tried_assignments = set()

    # Try to assign a counselor with the fewest total assignments of the current period type
    for counselor in sorted(counselors, key=lambda c: ((rest_count[c] if period_type == 'R' else night_count[c]), total_count[c])):
        if counselor in tried_assignments:
            continue
        can_assign_result, reason = can_assign(counselor, period, day, assignments)
        if can_assign_result:
            assignments[counselor].append(period)
            if period_type == 'R':
                rest_count[counselor] += 1
            else:
                night_count[counselor] += 1
            total_count[counselor] += 1

            if assign_periods(periods, assignments, counselors, rest_count, night_count, total_count, period_index + 1):
                return True

            assignments[counselor].remove(period)
            if period_type == 'R':
                rest_count[counselor] -= 1
            else:
                night_count[counselor] -= 1
            total_count[counselor] -= 1

        tried_assignments.add(counselor)

    return False

# Function to balance assignments
def balance_assignments(assignments, total_count, desired_assignments):
    over_assigned = []
    under_assigned = []

    for counselor, count in total_count.items():
        if count > desired_assignments:
            over_assigned.append(counselor)
        elif count < desired_assignments:
            under_assigned.append(counselor)

    while over_assigned and under_assigned:
        over_counselor = over_assigned.pop()
        under_counselor = under_assigned.pop()

        periods_to_reassign = [period for period in assignments[over_counselor] if can_assign(under_counselor, period, int(period[1:]), assignments)[0]]
        if periods_to_reassign:
            period = periods_to_reassign[0]
            assignments[over_counselor].remove(period)
            assignments[under_counselor].append(period)
            total_count[over_counselor] -= 1
            total_count[under_counselor] += 1

            if total_count[over_counselor] > desired_assignments:
                over_assigned.append(over_counselor)
            if total_count[under_counselor] < desired_assignments:
                under_assigned.append(under_counselor)

# Function to assign counselors to periods
def assign_counselors(rest_night_periods, counselors_off_days):
    assignments = defaultdict(list)
    rest_count = defaultdict(int)
    night_count = defaultdict(int)
    total_count = defaultdict(int)
    counselors = list(counselors_off_days.keys())

    if not assign_periods(rest_night_periods, assignments, counselors, rest_count, night_count, total_count):
        print("Failed to assign all periods to counselors")

    desired_assignments = len(rest_night_periods) // len(counselors)
    balance_assignments(assignments, total_count, desired_assignments)

    return dict(assignments), total_count, rest_count, night_count

# Assign counselors
assignments, total_assignments, rest_count, night_count = assign_counselors(rest_night_periods, counselors_off_days)

# Display the assignments and total assignments
for counselor, periods in assignments.items():
    print(f"{counselor}: {', '.join(periods)} ({len(periods)} assignments, {rest_count[counselor]} rests, {night_count[counselor]} nights)")

# Check for overall assignments imbalance
max_assignments = max(total_assignments.values())
min_assignments = min(total_assignments.values())
if max_assignments - min_assignments > 0:
    print(f"Imbalance detected: Max assignments = {max_assignments}, Min assignments = {min_assignments}")
else:
    print("Assignments are balanced.\n")

# Create a mapping from periods to counselors
period_to_counselor = {}
for counselor, periods in assignments.items():
    for period in periods:
        period_to_counselor[period] = counselor

# Sort the periods
sorted_periods = sorted(period_to_counselor.keys(), key=lambda x: (int(x[1:]), '0' if x.startswith('R') else '1'))

# Display the final schedule in the desired format
print("\nFinal Schedule:")
for period in sorted_periods:
    print(f"{period}: {period_to_counselor[period]}")

# Tests
for counselor, periods in assignments.items():
    for period in periods:
        day = int(period[1:])
        # Check that counselors are not assigned to off days
        if day in counselors_off_days[counselor]:
            print(f"{counselor} is assigned to off day {day}")
        # Check that counselors are not assigned on the night before a day off
        if period.startswith('N') and (day + 1) in counselors_off_days[counselor]:
            print(f"{counselor} is off the day after (day {day + 1})")
        # Check if the counselor is already assigned to both rest and night shifts on the same day
        if f'R{day}' in periods and f'N{day}' in periods:
            print(f"{counselor} is already assigned to both rest and night shifts on day {day}")
        # Check if the counselor is already assigned to a night shift the day before or the day after
        if (f'N{day - 1}' in assignments[counselor] and f'N{day}' in assignments[counselor]) or (f'N{day + 1}' in assignments[counselor] and f'N{day}' in assignments[counselor]):
            print(f"{counselor} is already assigned to a night shift the day before or the day after day {day}")
print("Schedule looks good!")

Counselor A1: R1, N4, R5, N6, R11, N12, N14, R16, N18, R20 (10 assignments, 5 rests, 5 nights)
Counselor B1: N1, R3, R6, N7, R10, N13, R15, N16, R19 (9 assignments, 5 rests, 4 nights)
Counselor C1: R2, N3, N5, N8, R9, N10, R14, R17, N19 (9 assignments, 4 rests, 5 nights)
Counselor C2: N2, R4, R8, N9, N11, R12, N15, N17, R18 (9 assignments, 4 rests, 5 nights)
Imbalance detected: Max assignments = 10, Min assignments = 9

Final Schedule:
R1: Counselor A1
N1: Counselor B1
R2: Counselor C1
N2: Counselor C2
R3: Counselor B1
N3: Counselor C1
R4: Counselor C2
N4: Counselor A1
R5: Counselor A1
N5: Counselor C1
R6: Counselor B1
N6: Counselor A1
N7: Counselor B1
R8: Counselor C2
N8: Counselor C1
R9: Counselor C1
N9: Counselor C2
R10: Counselor B1
N10: Counselor C1
R11: Counselor A1
N11: Counselor C2
R12: Counselor C2
N12: Counselor A1
N13: Counselor B1
R14: Counselor C1
N14: Counselor A1
R15: Counselor B1
N15: Counselor C2
R16: Counselor A1
N16: Counselor B1
R17: Counselor C1
N17: Counselor C2
R

# OD Scheduler (4+)

### This is a Python script that will create an on-duty scheduler for a double cabin with a multi-person rotation.

For a multi-person rotation, there will be two cabins called Cabin O (Odd) and Cabin E (Even).

Before running the script, the user will need to input the rest_night_periods. This is a list of rest and night periods where rests are differentiated by the letter R and nights are differentiated by the letter N. The numbers represent the number day of a session. For instance R1 would represent the rest period on the first day of the session. If a given day does not have a rest period, for instance day 7 might be a trip day and only have a night period, you would just skip R7 in the list and input N7 only. Similarily, if a night is an ALL-ON night, you would skip that night in the list. The nights are differentiated for each cabin with an O and E at the end. Each night should have an O and an E version (ie N1O and N1E).

Once the rest_night_periods are input, the user will need to adjust the counselors_off_days. This is a dictionary where the keys are the counselors and the values are a list of numbers which represent the days off a counselor has. For instance if counselor A1 is off on day 3 and day 10 in the session, the dictionary would contain "Counselor A1" : [3, 10]. Counselors in Cabin O (Odd) should only have odd number suffixes (Counselor A1, Counselor A3, etc). Counselors in Cabin E (Even) should only have even number suffixes (Counselor A2, Counselor A4). For example, if Cabin O has three counselors with off days as ACC, you would label those counselors as Counselor A1, Counselor C1, and Counselor C3 even if there are not 3 C-day off counselors. The numbering of the counselors is simply used to differentiated between odd and even cabins.

The user will also need to input the rest_counselors, night_counselors_O, and night_counselors_E. rest_counselors will be a list of all the counselors who can be scheduled for rest duties. night_counselors_O will be a list of all the cabin O counselors. These should be all the counselors that end with an Odd number. night_counselors_E will be a list of all the cabin E counselors. These should be all the counselors that end with an Even number.

After inputting the rest_night_periods, counslors_off_days, rest_counselors, night_counselors_O, and night_counselors_E, you can click the triangle play symbol in the upper right-hand corner of the code box to run the script.

The script will create a printout at the bottom of the code box. The top of the printout will calculate how many rests, nights per cabin, and total assignments a counselor has. It will flag if one counselor has significantly more assignments than others in the cabin. Odd counselors should have no night assignments in the even cabin and vice versa.

The script will also create a "Final Schedule" that will assign a counselor to each rest and night period in the session where night assignments in each cabin are differentiated by the suffix O or E. You will need this Final Schedule to create the pattern templates at the top of the OD schedule for the DHs.

In [3]:
from collections import defaultdict

# User Input
# Schedule of rest and night periods where the number is the number day of a session
rest_night_periods = ['R1', 'N1O','N1E', 'R2', 'N2O', 'N2E', 'R3', 'N3O', 'N3E', 'R4', 'N4O', 'N4E', 'R5', 'N5O', 'N5E',
                      'R6', 'N6O', 'N7O', 'N7E', 'R8', 'N8O', 'N8E', 'R9', 'N9O', 'N9E', 'R10', 'N10O', 'N10E',
                      'R11', 'N11O', 'N11E', 'R12', 'N12O', 'N13O', 'N13E', 'R14', 'N14O', 'N14E',
                      'R15', 'N15O', 'N15E', 'R16', 'N16O', 'N16E', 'R17', 'N17O', 'N17E', 'R18', 'N18O', 'N18E',
                      'R19', 'N19O', 'N19E', 'R20']

# List of days that counselors are not on duty
# Differentiate counselors that have the same off days by 1 or 2
counselors_off_days = {
    'Counselor A1': [3, 10],
    'Counselor B1': [5, 12],
    'Counselor B2': [5, 12],
    'Counselor C1': [7, 13],
    'Counselor C2': [7, 13],
    'Counselor C3': [7, 13],
    'Counselor C4': [7, 13]
}

# 7 counselors rotating through rest periods
rest_counselors = ['Counselor A1', 'Counselor B1', 'Counselor B2', 'Counselor C1', 'Counselor C2', 'Counselor C3', 'Counselor C4']

# 4 counselors for night shifts in cabin O
night_counselors_O = ['Counselor A1', 'Counselor B1', 'Counselor C1', 'Counselor C3']

# 3 counselors for night shifts in cabin E
night_counselors_E = ['Counselor B2', 'Counselor C2', 'Counselor C4']

# Function to check if a counselor can be assigned to a specific period
def can_assign(counselor, period, day, assignments):
    if day in counselors_off_days[counselor]:
        return False, f"{counselor} is off on day {day}"
    if period.startswith('N') and (day + 1) in counselors_off_days[counselor]:
        return False, f"{counselor} is off the day after (day {day + 1})"
    if f'R{day}' in assignments[counselor] or any(f'N{day}' in p for p in assignments[counselor]):
        return False, f"{counselor} is already assigned to a period on day {day}"
    # Check if the counselor is already assigned to a night shift the day before or the day after
    if period.startswith('N') and (f'N{day - 1}' in assignments[counselor] or f'N{day + 1}' in assignments[counselor]):
        return False, f"{counselor} is already assigned to a night shift the day before or the day after day {day}"
    # Check if the counselor is already assigned to a night shift on the same day but different cabin
    if period.startswith('N') and any(f'N{day}' in p for p in assignments[counselor]):
        return False, f"{counselor} is already assigned to a night shift on day {day}"
    # Check for back-to-back night shifts
    if period.startswith('N') and (f'N{day - 1}O' in assignments[counselor] or f'N{day - 1}E' in assignments[counselor] or f'N{day + 1}O' in assignments[counselor] or f'N{day + 1}E' in assignments[counselor]):
        return False, f"{counselor} is assigned to back-to-back night shifts on day {day}"
    return True, ""

# Backtracking function to assign counselors to periods
def assign_periods(periods, assignments, rest_counselors, night_counselors_O, night_counselors_E, rest_count, night_count_O, night_count_E, total_count, period_index=0):
    if period_index == len(periods):
        return True

    period = periods[period_index]
    day = int(''.join(filter(str.isdigit, period)))
    period_type = 'R' if period.startswith('R') else ('O' if period.endswith('O') else 'E')

    if period_type == 'R':
        counselors = rest_counselors
        count = rest_count
    elif period_type == 'O':
        counselors = night_counselors_O
        count = night_count_O
    else:
        counselors = night_counselors_E
        count = night_count_E

    for counselor in sorted(counselors, key=lambda c: (count[c], total_count[c])):
        can_assign_result, reason = can_assign(counselor, period, day, assignments)
        if can_assign_result:
            assignments[counselor].append(period)
            count[counselor] += 1
            total_count[counselor] += 1

            if assign_periods(periods, assignments, rest_counselors, night_counselors_O, night_counselors_E, rest_count, night_count_O, night_count_E, total_count, period_index + 1):
                return True

            assignments[counselor].remove(period)
            count[counselor] -= 1
            total_count[counselor] -= 1

    return False

# Function to assign counselors to periods
def assign_counselors(rest_night_periods, counselors_off_days):
    assignments = defaultdict(list)
    rest_count = defaultdict(int)
    night_count_O = defaultdict(int)
    night_count_E = defaultdict(int)
    total_count = defaultdict(int)

    if not assign_periods(rest_night_periods, assignments, rest_counselors, night_counselors_O, night_counselors_E, rest_count, night_count_O, night_count_E, total_count):
        print("Failed to assign all periods to counselors")

    return dict(assignments), rest_count, night_count_O, night_count_E, total_count

# Assign counselors
assignments, rest_count, night_count_O, night_count_E, total_assignments = assign_counselors(rest_night_periods, counselors_off_days)

# Create a mapping from periods to counselors
period_to_counselor = {}
for counselor, periods in assignments.items():
    for period in periods:
        period_to_counselor[period] = counselor

# Sort the periods
sorted_periods = sorted(period_to_counselor.keys(), key=lambda x: (int(''.join(filter(str.isdigit, x))), '0' if x.startswith('R') else '1', x))

# Display the count of rest and night shifts for each counselor
print("\nCounselor Assignments:")
for counselor in sorted(rest_count.keys()):
    print(f"{counselor}: {rest_count[counselor]} rests, {night_count_O[counselor]} nights in Cabin O, {night_count_E[counselor]} nights in Cabin E")

# Display the final schedule in the desired format
print("\nFinal Schedule:")
for period in sorted_periods:
    print(f"{period}: {period_to_counselor[period]}")


# Tests
for counselor, periods in assignments.items():
    for period in periods:
        day = int(''.join(filter(str.isdigit, period)))
        # Check that counselors are not assigned to off days
        if day in counselors_off_days[counselor]:
            print(f"{counselor} is assigned to off day {day}")
        # Check that counselors are not assigned on the night before a day off
        if period.startswith('N') and (day + 1) in counselors_off_days[counselor]:
            print(f"{counselor} is off the day after (day {day + 1})")
        # Check if the counselor is already assigned to both rest and night shifts on the same day
        if f'R{day}' in periods and f'N{day}' in periods:
            print(f"{counselor} is already assigned to both rest and night shifts on day {day}")
        # Check if the counselor is already assigned to a night shift the day before or the day after
        if (f'N{day - 1}' in assignments[counselor] and f'N{day}' in assignments[counselor]) or (f'N{day + 1}' in assignments[counselor] and f'N{day}' in assignments[counselor]):
            print(f"{counselor} is already assigned to a night shift the day before or the day after day {day}")
print("Schedule looks good!")


Counselor Assignments:
Counselor A1: 3 rests, 5 nights in Cabin O, 0 nights in Cabin E
Counselor B1: 3 rests, 5 nights in Cabin O, 0 nights in Cabin E
Counselor B2: 2 rests, 0 nights in Cabin O, 6 nights in Cabin E
Counselor C1: 3 rests, 4 nights in Cabin O, 0 nights in Cabin E
Counselor C2: 2 rests, 0 nights in Cabin O, 6 nights in Cabin E
Counselor C3: 3 rests, 5 nights in Cabin O, 0 nights in Cabin E
Counselor C4: 2 rests, 0 nights in Cabin O, 5 nights in Cabin E

Final Schedule:
R1: Counselor A1
N1E: Counselor B2
N1O: Counselor B1
R2: Counselor C1
N2E: Counselor C2
N2O: Counselor C3
R3: Counselor C4
N3E: Counselor B2
N3O: Counselor C1
R4: Counselor B1
N4E: Counselor C4
N4O: Counselor A1
R5: Counselor C3
N5E: Counselor C2
N5O: Counselor C1
R6: Counselor B2
N6O: Counselor A1
N7E: Counselor B2
N7O: Counselor B1
R8: Counselor C2
N8E: Counselor C4
N8O: Counselor C3
R9: Counselor A1
N9E: Counselor C2
N9O: Counselor B1
R10: Counselor C1
N10E: Counselor C4
N10O: Counselor C3
R11: Counselo